In [1]:
import json
import os
import pandas as pd
import re

C:\Users\Thorn\AppData\Local\Temp\ipykernel_2868\819515313.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import spacy
from spacy import displacy
from spacy.tokens import Span, Doc

D:\Anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import TFDebertaForTokenClassification, DebertaTokenizer

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
from scipy import stats
import numpy as np

In [6]:
# import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch

In [7]:
import torch

# Load Data

In [8]:
path_train = './in/train.json'
path_test = './in/test.json'

In [9]:
train_json = json.load(open(path_train))
df_train = pd.json_normalize(train_json)

test_json = json.load(open(path_test))
df_test = pd.json_normalize(test_json)

In [10]:
train_json[0].keys()

dict_keys(['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels'])

In [11]:
df_train.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [12]:
text_test = """Meet Jane Doe, a brilliant student at XYZ University. She can be reached at jane.doe@email.com or through her phone number +1234567890. Jane resides at 123 Main Street, Cityville. Her student ID is 987654 and her personal website is www.janedoe.com. Connect with her on social media using the username @janedoe.

Meanwhile, John Smith, another outstanding student, can be contacted at john.smith@email.com or at +9876543210. John lives at 456 Oak Avenue, Townsville. His student ID is 123456, and you can visit his personal blog at www.johnsmithblog.com. Follow him on Twitter with the handle @johnsmith123.

For any inquiries about the university's programs, you can contact the administration office at admin@xyzuniversity.edu or call +5551234567. The office is located at 789 University Boulevard.

Visit our official website at www.xyzuniversity.edu for more information on courses and admission procedures.

"""


In [13]:
text_test

"Meet Jane Doe, a brilliant student at XYZ University. She can be reached at jane.doe@email.com or through her phone number +1234567890. Jane resides at 123 Main Street, Cityville. Her student ID is 987654 and her personal website is www.janedoe.com. Connect with her on social media using the username @janedoe.\n\nMeanwhile, John Smith, another outstanding student, can be contacted at john.smith@email.com or at +9876543210. John lives at 456 Oak Avenue, Townsville. His student ID is 123456, and you can visit his personal blog at www.johnsmithblog.com. Follow him on Twitter with the handle @johnsmith123.\n\nFor any inquiries about the university's programs, you can contact the administration office at admin@xyzuniversity.edu or call +5551234567. The office is located at 789 University Boulevard.\n\nVisit our official website at www.xyzuniversity.edu for more information on courses and admission procedures.\n\n"

In [14]:
classes = ['O',
'B-EMAIL',
'B-ID_NUM',
'B-NAME_STUDENT',
'B-PHONE_NUM',
'B-STREET_ADDRESS',
'B-URL_PERSONAL',
'B-USERNAME',
'I-ID_NUM',
'I-NAME_STUDENT',
'I-PHONE_NUM',
'I-STREET_ADDRESS',
'I-URL_PERSONAL',
]

classes2id = {clas:i for i, clas in enumerate(classes)}


In [15]:
classes_pos = classes[1:]

# Model

## Quick Pipeline

In [16]:
from transformers import pipeline
gen = pipeline("token-classification", "lakshyakh93/deberta_finetuned_pii", device=-1)

# temp_text = "My name is John and I live in California."
temp_text = text_test
output = gen(temp_text, aggregation_strategy="first")

D:\Anaconda3\envs\nlp\lib\site-packages\transformers\pipelines\token_classification.py:393: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


In [17]:
text_test

"Meet Jane Doe, a brilliant student at XYZ University. She can be reached at jane.doe@email.com or through her phone number +1234567890. Jane resides at 123 Main Street, Cityville. Her student ID is 987654 and her personal website is www.janedoe.com. Connect with her on social media using the username @janedoe.\n\nMeanwhile, John Smith, another outstanding student, can be contacted at john.smith@email.com or at +9876543210. John lives at 456 Oak Avenue, Townsville. His student ID is 123456, and you can visit his personal blog at www.johnsmithblog.com. Follow him on Twitter with the handle @johnsmith123.\n\nFor any inquiries about the university's programs, you can contact the administration office at admin@xyzuniversity.edu or call +5551234567. The office is located at 789 University Boulevard.\n\nVisit our official website at www.xyzuniversity.edu for more information on courses and admission procedures.\n\n"

In [18]:
output[16:]

[{'entity_group': 'ZIPCODE',
  'score': 0.59202105,
  'word': ' 123456,',
  'start': 484,
  'end': 492},
 {'entity_group': 'URL',
  'score': 0.72377187,
  'word': ' www.johnsmithblog.com.',
  'start': 531,
  'end': 554},
 {'entity_group': 'EMAIL',
  'score': 0.9881231,
  'word': ' admin@xyzuniversity.edu',
  'start': 704,
  'end': 728},
 {'entity_group': 'PHONE_NUMBER',
  'score': 0.9487184,
  'word': ' +5551234567.',
  'start': 736,
  'end': 749},
 {'entity_group': 'STREETADDRESS',
  'score': 0.9893532,
  'word': ' 789 University Boulevard.\n\nVisit',
  'start': 774,
  'end': 807},
 {'entity_group': 'URL',
  'score': 0.95440745,
  'word': ' www.xyzuniversity.edu',
  'start': 831,
  'end': 853}]

## Yanis

In [19]:
model_name = "Yanis/microsoft-deberta-v3-large_ner_conll2003-anonimization_TRY_1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

In [20]:
model_labels = model.config.id2label

print(model_labels)

{0: 'O', 1: 'Racial or ethnic information', 2: 'Name', 3: 'Sexual orientation', 4: 'Phone Numbers', 5: 'Health insurance information', 6: 'Religious beliefs', 7: 'Political opinions', 8: 'Social Security number', 9: 'Civil state', 10: 'Medical record numbers', 11: 'Trade union membership', 12: 'Physical addresses', 13: 'Philosophical beliefs', 14: 'Email address', 15: 'Genetic information', 16: 'Financial information', 17: 'Biometric infomration'}


In [21]:
yanis_to_cur = {
    0:'O', 
    1:'O',
    2:'NAME_STUDENT',
    3:'O',
    4:'PHONE_NUM',
    5:'O',
    6:'O',
    7:'O',
    8:'ID_NUM',
    9:'O',
    10:'ID_NUM',
    11:'O',
    12:'STREET_ADDRESS',
    13:'O',
    14:'EMAIL',
    15:'O',
    16:'O',
    17:'O'}

# Experiment

## Spacy tokenizer

# Parts

## Token Limit (temp)

In [24]:
full_text = df_train.loc[0].full_text
nlp = spacy.load("en_core_web_sm")
doc = nlp(full_text)
text_orig = ''.join([token.text_with_ws for token in doc[:]])
text_adj = text_orig.replace('\n', ' ')

In [110]:
text

"Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla\n\nChallenge & selection\n\nThe tool I use to help all stakeholders finding their way through the complexity of a project is the  mind map.\n\nWhat exactly is a mind map? According to the definition of Buzan T. and Buzan B. (1999, Dessine-moi  l'intelligence. Paris: Les Éditions d'Organisation.), the mind map (or heuristic diagram) is a graphic  representation technique that follows the natural functioning of the mind and allows the brain's  potential to be released. Cf Annex1\n\nThis tool has many advantages:\n\n•  It is accessible to all and does not require significant material investment and can be done  quickly\n\n•  It is scalable\n\n•  It allows categorization and linking of information\n\n•  It can be applied to any type of situation: notetaking, problem solving, analysis, creation of  new ideas\n\n•  It is suitable for all people and is easy to learn\n\n•  It is fun and encourages "

In [111]:
text_adj

"Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla  Challenge & selection  The tool I use to help all stakeholders finding their way through the complexity of a project is the  mind map.  What exactly is a mind map? According to the definition of Buzan T. and Buzan B. (1999, Dessine-moi  l'intelligence. Paris: Les Éditions d'Organisation.), the mind map (or heuristic diagram) is a graphic  representation technique that follows the natural functioning of the mind and allows the brain's  potential to be released. Cf Annex1  This tool has many advantages:  •  It is accessible to all and does not require significant material investment and can be done  quickly  •  It is scalable  •  It allows categorization and linking of information  •  It can be applied to any type of situation: notetaking, problem solving, analysis, creation of  new ideas  •  It is suitable for all people and is easy to learn  •  It is fun and encourages exchanges  •  It makes visible the dimension of p

## Instantiate model

In [22]:
model_name = "Yanis/microsoft-deberta-v3-large_ner_conll2003-anonimization_TRY_1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

## Tokenization -> Inference

In [25]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
import spacy

# Load pre-trained BERT model and tokenizer using Auto classes
# model_name = "lakshyakh93/deberta_finetuned_pii"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForTokenClassification.from_pretrained(model_name)

# Load spaCy model for tokenization
nlp = spacy.load("en_core_web_sm")

# Sample text
# text = "John Smith works at OpenAI in San Francisco."
# text = df_train.loc[0].full_text
text = text_orig # token_limit
# text = text_adj # no next line
# text = text_test

doc = nlp(text)

# Step 1: Tokenize the text using spaCy for words
words = [token.text for token in doc]

# Step 2: Tokenize the text using the model's tokenizer and get word to subword mapping
# inputs = tokenizer(text, return_tensors='pt', return_offsets_mapping=False)
inputs = tokenizer(words, return_tensors='pt', return_offsets_mapping=False, is_split_into_words=True)
word_ids = inputs.word_ids()

In [26]:
%%time
# Step 3: Run inference
with torch.no_grad():
    outputs = model(**inputs).logits

CPU times: total: 6min 50s
Wall time: 3min 59s


In [114]:
# tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

## Highest non-O probability:

In [115]:
# Function to get the highest non-'O' labels for each token with a threshold
def get_labels(tokens, label_probabilities, threshold=0.5):
    highest_non_O_labels = []

    for token, probabilities in zip(tokens, label_probabilities):
        non_O_labels = []
        non_O_probabilities = []

        # Iterate through predicted label probabilities
        for label_idx, probability in enumerate(probabilities):
            label_name = model.config.id2label[label_idx]
            # Exclude 'O' labels and labels with probabilities below the threshold
            if label_name != 'O' and probability >= threshold:
                non_O_labels.append(label_name)
                non_O_probabilities.append(probability)

        # If there are non-'O' labels above the threshold, select the one with the highest probability
        if non_O_labels:
            highest_non_O_label = non_O_labels[non_O_probabilities.index(max(non_O_probabilities))]
            highest_non_O_labels.append((token, highest_non_O_label))
        else:
            # If all labels are 'O' or below the threshold, consider it as non-entity
            highest_non_O_labels.append((token, 'O'))

    return highest_non_O_labels

In [168]:
label_probabilities = torch.softmax(outputs, dim=2)[0].cpu().numpy()
tokens = inputs.tokens()

In [117]:
threshold = 0.1

# Get highest non-'O' labels for each token with thresholding
token_labels = get_labels(tokens, label_probabilities, threshold)

pred_labels = np.array([label for token,label in token_labels])

## Prediction Alignment

In [201]:
# Step 4: Create a list of which tokens or subwords correspond to a word using the word_ids variable
word_subword_mapping = {}
for i, word_id in enumerate(word_ids):
    if word_id is not None:
        if word_id not in word_subword_mapping:
            word_subword_mapping[word_id] = []
        word_subword_mapping[word_id].append(i)
        
        
def get_majority(arr):
    unique_elements, counts = np.unique(arr, return_counts=True)
    max_count_index = np.argmax(counts)
    return unique_elements[max_count_index]

        
# Step 5: Iterate through pairs of words and subwords to count the majority label
word_labels = []
# for word_id, subword_indices in word_subword_mapping.items():
#     subword_labels = pred_labels[subword_indices]
#     majority_label = get_majority(subword_labels)
#     word_labels.append((words[word_id], majority_label))
    
for i, word in enumerate(words):
    if i in word_subword_mapping:
        subword_labels = pred_labels[word_subword_mapping[i]]
        majority_label = get_majority(subword_labels)
        word_labels.append((word, majority_label))
    else:
        word_labels.append((word,'O'))

# Display results
# print("Original Text:", text)
# print("Word Labels:", word_labels)

## Reconstruct (Visualize)

In [202]:
def visualize_span(full_text, tokens, labels):
    
    global span_infos
    
    start_pos = -1
    span_infos = []
    for label_index, label in enumerate(labels):
        if re.match('^B-', label):
            start_pos = label_index
            label_group = label.split('-')[1]

        if re.match('O', label) and start_pos != -1:
            end_pos = label_index
            span_dict = {'start_pos':start_pos, 'end_pos':end_pos, 'label_group':label_group}
            span_infos.append(span_dict)
            start_pos = -1

    doc_spans = []
    nlp = spacy.blank('en')
    doc = nlp(full_text)
    for span_info in span_infos:
        _span = Span(doc, span_info['start_pos'], span_info['end_pos'], span_info['label_group'])
        doc_spans.append(_span)

    doc.spans['sc'] = doc_spans

    displacy.render(doc, style = 'span')

In [203]:
def visualize_label(full_text, tokens, labels):
    global doc, span_infos, doc_spans
    
    start_pos = -1
    span_infos = []
    for label_index, label in enumerate(labels):
        if label!= 'O':
            start_pos = label_index
            end_pos = start_pos + 1
            span_dict = {'start_pos':start_pos, 'end_pos':end_pos, 'label':label}
            span_infos.append(span_dict)

    doc_spans = []
#     nlp = spacy.blank('en')
#     doc = nlp(full_text)
    doc = Doc(nlp.vocab, words=tokens)
    
    for span_info in span_infos:
        _span = Span(doc, span_info['start_pos'], span_info['end_pos'], span_info['label'])
        doc_spans.append(_span)

    doc.spans['sc'] = doc_spans
    
#     color_map = {'B-EMAIL': '#2fc3da',
#                  'B-ID_NUM': '#1c0cfa',
#                  'B-NAME_STUDENT':'#e01d82',
#                  'B-PHONE_NUM': '#ebe70e',
#                  'B-STREET_ADDRESS':'#f2860a',
#                  'B-URL_PERSONAL': '#c9f211',
#                  'B-USERNAME': '#0eebb7',
#                  'I-ID_NUM': '#8e87ed',
#                  'I-NAME_STUDENT':'#eb83b9',
#                  'I-PHONE_NUM': '#e6e4a1',
#                  'I-STREET_ADDRESS': '#f2c694',
#                  'I-URL_PERSONAL':'#e5f2ac'}
#     options = {'colors': color_map}

#     displacy.render(doc, style = 'span', options = options)

    displacy.render(doc, style = 'span')

### Result

In [204]:
text

"Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla\n\nChallenge & selection\n\nThe tool I use to help all stakeholders finding their way through the complexity of a project is the  mind map.\n\nWhat exactly is a mind map? According to the definition of Buzan T. and Buzan B. (1999, Dessine-moi  l'intelligence. Paris: Les Éditions d'Organisation.), the mind map (or heuristic diagram) is a graphic  representation technique that follows the natural functioning of the mind and allows the brain's  potential to be released. Cf Annex1\n\nThis tool has many advantages:\n\n•  It is accessible to all and does not require significant material investment and can be done  quickly\n\n•  It is scalable\n\n•  It allows categorization and linking of information\n\n•  It can be applied to any type of situation: notetaking, problem solving, analysis, creation of  new ideas\n\n•  It is suitable for all people and is easy to learn\n\n•  It is fun and encourages exchanges\n\n•  It makes visi

In [205]:
token_v, pred_v = zip(*word_labels)

visualize_span(text, token_v, pred_v)

In [206]:
visualize_label(text, token_v, pred_v)

### Debug

In [98]:
pred_v[:15]

('O',
 'O',
 'O',
 'O',
 'B-FIRSTNAME',
 'I-FIRSTNAME',
 'I-FIRSTNAME',
 'B-NUMBER',
 'I-NUMBER',
 'I-NUMBER',
 'I-NUMBER',
 'O',
 'O',
 'O',
 'O')

In [74]:
span_infos

[{'start_pos': 2, 'end_pos': 3, 'label_group': 'MIDDLENAME'},
 {'start_pos': 24, 'end_pos': 25, 'label_group': 'FIRSTNAME'},
 {'start_pos': 27, 'end_pos': 30, 'label_group': 'STREETADDRESS'},
 {'start_pos': 31, 'end_pos': 32, 'label_group': 'CITY'},
 {'start_pos': 37, 'end_pos': 38, 'label_group': 'ZIPCODE'},
 {'start_pos': 54, 'end_pos': 55, 'label_group': 'MIDDLENAME'},
 {'start_pos': 60, 'end_pos': 61, 'label_group': 'LASTNAME'},
 {'start_pos': 75, 'end_pos': 76, 'label_group': 'FIRSTNAME'},
 {'start_pos': 78, 'end_pos': 81, 'label_group': 'STREETADDRESS'},
 {'start_pos': 82, 'end_pos': 83, 'label_group': 'CITY'},
 {'start_pos': 88, 'end_pos': 89, 'label_group': 'ZIPCODE'},
 {'start_pos': 107, 'end_pos': 108, 'label_group': 'USERNAME'},
 {'start_pos': 136, 'end_pos': 139, 'label_group': 'STREETADDRESS'}]

In [ ]:
list(zip(*word_labels))

In [112]:
text

"Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla\n\nChallenge & selection\n\nThe tool I use to help all stakeholders finding their way through the complexity of a project is the  mind map.\n\nWhat exactly is a mind map? According to the definition of Buzan T. and Buzan B. (1999, Dessine-moi  l'intelligence. Paris: Les Éditions d'Organisation.), the mind map (or heuristic diagram) is a graphic  representation technique that follows the natural functioning of the mind and allows the brain"

In [116]:
token_v[:15]

('Design',
 'Thinking',
 'for',
 'innovation',
 'reflexion',
 '-',
 'Avril',
 '2021',
 '-',
 'Nathalie',
 'Sylla',
 '\n\n',
 'Challenge',
 '&',
 'selection')

In [23]:
pred_v[:15]

('O',
 'O',
 'O',
 'O',
 'B-FIRSTNAME',
 'O',
 'I-FIRSTNAME',
 'B-NUMBER',
 'I-NUMBER',
 'I-NUMBER',
 'I-NUMBER',
 'O',
 'O',
 'O',
 'O')

In [118]:
word_subword_mapping

{0: [1],
 1: [2],
 2: [3],
 3: [4],
 4: [5, 6],
 5: [7],
 6: [8, 9],
 7: [10],
 8: [11],
 9: [12, 13, 14, 15],
 10: [16, 17],
 11: [18],
 12: [19],
 13: [20, 21],
 14: [22],
 15: [23],
 16: [24],
 17: [25],
 18: [26],
 19: [27],
 20: [28],
 21: [29],
 22: [30],
 23: [31],
 24: [32],
 25: [33],
 26: [34],
 27: [35],
 28: [36],
 29: [37],
 30: [38],
 31: [39],
 32: [40],
 33: [41],
 34: [42],
 35: [43],
 36: [44],
 37: [45],
 38: [46],
 39: [47],
 40: [48],
 41: [49],
 42: [50],
 43: [51],
 44: [52],
 45: [53],
 46: [54],
 47: [55],
 48: [56],
 49: [57],
 50: [58],
 51: [59],
 52: [60],
 53: [61],
 54: [62],
 55: [63, 64],
 56: [65],
 57: [66],
 58: [67],
 59: [68, 69],
 60: [70],
 61: [71],
 62: [72],
 63: [73],
 64: [74],
 65: [75, 76, 77],
 66: [78],
 67: [79, 80],
 68: [81],
 69: [82],
 70: [83],
 71: [84],
 72: [85],
 73: [86],
 74: [87],
 75: [88],
 76: [89, 90, 91],
 77: [92],
 78: [93],
 79: [94, 95],
 80: [96],
 81: [97],
 82: [98],
 83: [99],
 84: [100],
 85: [101],
 86: [102, 

In [99]:
word_labels

[('Meet', 'O'),
 ('Jane', 'B-FIRSTNAME'),
 ('Doe', 'O'),
 (',', 'O'),
 ('a', 'O'),
 ('brilliant', 'O'),
 ('student', 'O'),
 ('at', 'O'),
 ('XYZ', 'O'),
 ('University', 'O'),
 ('.', 'O'),
 ('She', 'O'),
 ('can', 'O'),
 ('be', 'O'),
 ('reached', 'O'),
 ('at', 'O'),
 ('jane.doe@email.com', 'I-EMAIL'),
 ('or', 'O'),
 ('through', 'O'),
 ('her', 'O'),
 ('phone', 'O'),
 ('number', 'O'),
 ('+1234567890', 'I-PHONE_NUMBER'),
 ('.', 'O'),
 ('Jane', 'B-FIRSTNAME'),
 ('resides', 'O'),
 ('at', 'O'),
 ('123', 'B-STREETADDRESS'),
 ('Main', 'I-STREETADDRESS'),
 ('Street', 'I-STREETADDRESS'),
 (',', 'O'),
 ('Cityville', 'B-CITY'),
 ('.', 'O'),
 ('Her', 'O'),
 ('student', 'O'),
 ('ID', 'O'),
 ('is', 'O'),
 ('987654', 'B-ZIPCODE'),
 ('and', 'O'),
 ('her', 'O'),
 ('personal', 'O'),
 ('website', 'O'),
 ('is', 'O'),
 ('www.janedoe.com', 'I-URL'),
 ('.', 'O'),
 ('Connect', 'O'),
 ('with', 'O'),
 ('her', 'O'),
 ('on', 'O'),
 ('social', 'O'),
 ('media', 'O'),
 ('using', 'O'),
 ('the', 'O'),
 ('username', 'O'),


In [98]:
span_infos

[{'start_pos': 1, 'end_pos': 2, 'label_group': 'FIRSTNAME'},
 {'start_pos': 24, 'end_pos': 25, 'label_group': 'FIRSTNAME'},
 {'start_pos': 27, 'end_pos': 30, 'label_group': 'STREETADDRESS'},
 {'start_pos': 31, 'end_pos': 32, 'label_group': 'CITY'},
 {'start_pos': 37, 'end_pos': 38, 'label_group': 'ZIPCODE'},
 {'start_pos': 54, 'end_pos': 55, 'label_group': 'MIDDLENAME'},
 {'start_pos': 60, 'end_pos': 61, 'label_group': 'LASTNAME'},
 {'start_pos': 78, 'end_pos': 81, 'label_group': 'STREETADDRESS'},
 {'start_pos': 82, 'end_pos': 83, 'label_group': 'CITY'},
 {'start_pos': 88, 'end_pos': 89, 'label_group': 'ZIPCODE'},
 {'start_pos': 107, 'end_pos': 108, 'label_group': 'USERNAME'},
 {'start_pos': 136, 'end_pos': 139, 'label_group': 'STREETADDRESS'}]

## Map labels to new labels

In [207]:
pred_conv = [yanis_to_cur[model.config.label2id[pred]] for pred in pred_v]

### Vis

In [208]:
visualize_label(text, token_v, pred_conv)

### To BIO

In [209]:
def convert_to_bio(tokens, labels):
    bio_labels = []
    current_entity = None
    
    for token, label in zip(tokens, labels):
        if label == 'O':
            bio_labels.append(label)
            current_entity = None
        else:
            if current_entity == label:
                bio_labels.append('I-' + label)
            else:
                bio_labels.append('B-' + label)
            current_entity = label
    
    return bio_labels

# Example usage:
tokens = ['Apple', 'Inc', 'is', 'a', 'company', 'based', 'in', 'California']
labels = ['ORG', 'ORG', 'O', 'O', 'O', 'O', 'O', 'LOC']

bio_labels = convert_to_bio(tokens, labels)
print(bio_labels)

['B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'B-LOC']


In [210]:
pred_bio = convert_to_bio(token_v, pred_conv)

In [211]:
visualize_label(text, token_v, pred_bio)

## Scoring

In [212]:
import torch
from sklearn.metrics import fbeta_score

def f_beta_score(predictions, labels, beta=5):
    """
    Compute the F-beta score for token classification task.
    
    Args:
    - predictions (torch.Tensor): Predicted labels
    - labels (torch.Tensor): True labels
    - beta (float): Determines the weight of precision in the combined score. 
                    beta < 1 lends more weight to precision, while beta > 1 favors recall.
    
    Returns:
    - f_beta (float): The computed F-beta score
    """
    predictions = predictions.flatten().cpu().numpy()
    labels = labels.flatten().cpu().numpy()

    # Set average='macro' to compute the F-beta score for each label without considering class imbalance
    return fbeta_score(labels, predictions, beta=beta, average='macro')

# Example usage
predictions = torch.tensor([[0, 1, 1, 2], [1, 2, 2, 0]])  # Example predicted labels
labels = torch.tensor([[0, 1, 1, 1], [0, 2, 2, 2]])       # Example true labels

f_beta = f_beta_score(predictions, labels, beta=5)
print("F-beta score:", f_beta)

F-beta score: 0.611111111111111


In [137]:
labels_true = df_train.loc[0].labels
labels_true[:20]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [213]:
pred_bio[:20]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [216]:
fbeta_score
f_beta = fbeta_score(labels_true, pred_bio, beta=5, average='macro')
print("F-beta score:", f_beta)

F-beta score: 0.9773745422081355


In [217]:
f_beta = fbeta_score(labels_true, pred_bio, beta=5, average='micro')
print("F-beta score:", f_beta)

F-beta score: 0.9933598937583001


In [220]:
f_beta = fbeta_score(labels_true, pred_bio, labels = classes_pos,beta=5, average='micro')
print("F-beta score:", f_beta)

F-beta score: 0.968944099378882


In [222]:
labels_torch = torch.Tensor([classes2id[label] for label in labels_true])
pred_torch = torch.Tensor([classes2id[pred] for pred in pred_bio])

In [224]:
def get_fbeta_score_multiclass(y_true, y_pred, beta=5):
    '''
    Parameters:
    - y_true: ground truth labels (shape: (batch_size))
    - y_pred: predicted labels (shape: (batch_size))
    - beta: beta value for F-beta score

    Returns:
    - float: micro F-beta multiclass score disregarding the background class (class 0)
    '''
    # Calculate FNFP both
    FNFP = torch.sum((y_pred != 0) & (y_true != 0) & (y_pred != y_true))
    # Calculate FP
    FP = torch.sum((y_pred != 0) & (y_true == 0)) + FNFP
    # Calculate FN
    FN = torch.sum((y_pred == 0) & (y_true != 0)) + FNFP
    # Calculate TP
    TP = torch.sum((y_pred != 0) & (y_true != 0) & (y_pred == y_true))
    
    # Calculate F-beta score
    fbeta = (1 + beta**2) * TP / ((1 + beta**2) * TP + beta**2 * FN + FP)
    return fbeta

In [226]:
score = get_fbeta_score_multiclass(labels_torch,pred_torch)

In [230]:
score.item()

0.9689440727233887

# Latest Pipeline

# Experiment

## Chunking

In [50]:
len(predicted_tokens_classes)

835

In [36]:
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))
tokens

['[CLS]',
 '▁Design',
 '▁Thinking',
 '▁for',
 '▁innovation',
 '▁reflex',
 'ion',
 '-',
 'Av',
 'ril',
 '▁2021',
 '-',
 'N',
 'atha',
 'lie',
 '▁S',
 'ylla',
 '▁Challenge',
 '▁&',
 '▁selection',
 '▁The',
 '▁tool',
 '▁I',
 '▁use',
 '▁to',
 '▁help',
 '▁all',
 '▁stakeholders',
 '▁finding',
 '▁their',
 '▁way',
 '▁through',
 '▁the',
 '▁complexity',
 '▁of',
 '▁a',
 '▁project',
 '▁is',
 '▁the',
 '▁mind',
 '▁map',
 '.',
 '▁What',
 '▁exactly',
 '▁is',
 '▁a',
 '▁mind',
 '▁map',
 '?',
 '▁According',
 '▁to',
 '▁the',
 '▁definition',
 '▁of',
 '▁Buz',
 'an',
 '▁T',
 '.',
 '▁and',
 '▁Buz',
 'an',
 '▁B',
 '.',
 '▁(',
 '1999',
 ',',
 '▁Des',
 's',
 'ine',
 '-',
 'moi',
 '▁l',
 "'",
 'intelligence',
 '.',
 '▁Paris',
 ':',
 '▁Les',
 '▁É',
 'dition',
 's',
 '▁d',
 "'",
 'Organ',
 'isation',
 '.',
 ')',
 ',',
 '▁the',
 '▁mind',
 '▁map',
 '▁(',
 'or',
 '▁heuristic',
 '▁diagram',
 ')',
 '▁is',
 '▁a',
 '▁graphic',
 '▁representation',
 '▁technique',
 '▁that',
 '▁follows',
 '▁the',
 '▁natural',
 '▁functioning',


In [50]:
word_label_mapping

{'your': tensor([0, 1, 1, 1, 0, 0, 1])}

In [54]:
filtered_words

['your', 'original', 'sentence', 'here']

In [52]:
word_label_mapping = dict(zip(filtered_words, predicted_labels))

In [53]:
word_label_mapping

{'your': tensor([0, 1, 1, 1, 0, 0, 1])}

In [49]:
filtered_words

['your', 'original', 'sentence', 'here']

## Original tokenizer

In [165]:
full_text = df_train.loc[0].full_text
tokens_orig = df_train.loc[0].tokens

In [151]:
token_v[:20]

('Design',
 'Thinking',
 'for',
 'innovation',
 'reflexion',
 '-',
 'Avril',
 '2021',
 '-',
 'Nathalie',
 'Sylla',
 'Challenge',
 '&',
 'selection',
 'The',
 'tool',
 'I',
 'use',
 'to',
 'help')

In [166]:
tokens_orig[:20]

['Design',
 'Thinking',
 'for',
 'innovation',
 'reflexion',
 '-',
 'Avril',
 '2021',
 '-',
 'Nathalie',
 'Sylla',
 '\n\n',
 'Challenge',
 '&',
 'selection',
 '\n\n',
 'The',
 'tool',
 'I',
 'use']

In [147]:
for i, token_orig in enumerate(tokens_orig):
    if token_orig != token_v[i]:
        print(f'{token_orig} != {token_v[i]}')



 != Challenge
Challenge != &
& != selection
selection != The


 != tool
The != I
tool != use
I != to
use != help
to != all
help != stakeholders
all != finding
stakeholders != their
finding != way
their != through
way != the
through != complexity
the != of
complexity != a
of != project
a != is
project != the
is != mind
the != map
  != .
mind != What
map != exactly
. != is


 != a
What != mind
exactly != map
is != ?
a != According
mind != to
map != the
? != definition
According != of
to != Buzan
the != T.
definition != and
of != Buzan
Buzan != B.
T. != (
and != 1999
Buzan != ,
B. != Dessine
( != -
1999 != moi
, != l'intelligence
Dessine != .
- != Paris
moi != :
  != Les
l'intelligence != Éditions
. != d'Organisation
Paris != .
: != )
Les != ,
Éditions != the
d'Organisation != mind
. != map
) != (
, != or
the != heuristic
mind != diagram
map != )
( != is
or != a
heuristic != graphic
diagram != representation
) != technique
is != that
a != follows
graphic != the
  != natural
representati

IndexError: tuple index out of range

# Old

## Pred alignment

In [22]:
# Step 4: Create a list of which tokens or subwords correspond to a word using the word_ids variable
word_subword_mapping = {}
for i, word_id in enumerate(word_ids):
    if word_id is not None:
        if word_id not in word_subword_mapping:
            word_subword_mapping[word_id] = []
        word_subword_mapping[word_id].append(i)

# Instantiate predictions variable
predictions = torch.argmax(outputs, dim=2)

# Step 5: Iterate through pairs of words and subwords to count the majority label
word_labels = []
for word_id, subword_indices in word_subword_mapping.items():
    subword_labels = predictions[0, subword_indices]
    majority_label = torch.mode(subword_labels).values.item()
    word_labels.append((words[word_id], model.config.id2label[majority_label]))

# Display results
# print("Original Text:", text)
# print("Word Labels:", word_labels)